In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [2]:
traindata = pd.read_csv("hr-total-cat-equalized.csv",index_col=0)

In [3]:
Y = traindata['is_promoted'].values
X_traindata = traindata.drop('is_promoted', axis=1)
X = X_traindata.values

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [5]:
from sklearn.tree import DecisionTreeClassifier

##Gini Index

In [6]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 42,
                               max_depth=7, min_samples_leaf=5)
clf_gini.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [7]:
y_pred = clf_gini.predict(X_test)
y_pred[0:20]

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
      dtype=int64)

In [8]:
df = pd.DataFrame(y_pred)

In [9]:
print("Accuracy is ", accuracy_score(Y_test,y_pred)*100)

Accuracy is  78.2703436450668


In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_pred)

array([[ 9863,  4458],
       [ 1625, 12048]], dtype=int64)

In [11]:
from sklearn.metrics import classification_report

In [12]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.69      0.76     14321
           1       0.73      0.88      0.80     13673

   micro avg       0.78      0.78      0.78     27994
   macro avg       0.79      0.78      0.78     27994
weighted avg       0.80      0.78      0.78     27994



In [13]:
from sklearn.model_selection import cross_val_score
cv_results = cross_val_score(clf_gini, X, Y, cv=10)
print(cv_results)

[0.78429061 0.78129018 0.7840763  0.79080484 0.7807309  0.78158825
 0.78233844 0.78298146 0.78791126 0.784589  ]


In [14]:
np.mean(cv_results)

0.7840601258483135

Finding out using test data given in the problem set

In [20]:
X_testdata = pd.read_csv("test-hr-total-cat1.csv",index_col=0)

In [21]:
X_testdata1 = X_testdata.values

In [22]:
final_prediction_unknown = clf_gini.predict(X_testdata)
final_prediction_unknown[0:20]

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
      dtype=int64)

In [23]:
final_prediction_unknown = final_prediction_unknown.tolist()
X_testdata['prediction']=final_prediction_unknown
final = X_testdata['prediction']

In [24]:
final

employee_id
8724     1
74430    0
72255    0
38562    0
64486    0
46232    0
54542    1
67269    0
66174    0
76303    1
60245    1
42639    0
30963    0
54055    0
42996    1
12737    1
27561    0
26622    1
31582    0
29793    0
72735    0
5677     0
60889    0
51498    0
8566     0
53151    0
16203    1
61083    1
74849    0
13259    0
        ..
61754    0
72190    1
27155    0
15959    1
72900    1
57683    1
55174    0
51115    0
50134    1
59775    1
14408    0
4351     0
59625    1
29379    0
67979    0
61451    0
73871    0
52090    0
18488    0
59673    1
39410    0
11837    1
78278    1
49291    0
47537    0
53478    0
25600    0
45409    0
1186     0
5973     1
Name: prediction, Length: 22456, dtype: int64

##Entropy Index

clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
 max_depth=7, min_samples_leaf=5)
clf_entropy.fit(X_train, Y_train)

y_pred_en = clf_entropy.predict(X_test)
y_pred_en[0:20]

print("Accuracy is ", accuracy_score(Y_test,y_pred_en)*100)

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_pred_en)

print(classification_report(Y_test,y_pred_en))

from sklearn.model_selection import cross_val_score
cv_results = cross_val_score(clf_entropy, X, Y, cv=10)
print(cv_results)

np.mean(cv_results)

X_testdata = pd.read_csv("test-data-coded.csv",index_col=0)

y_pred = clf_gini.predict(X_testdata)
y_pred[0:20]

pd.merge(X_testdata)